# 001 Staging

DB Connection

In [1]:
import yaml
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine, text
import pandas as pd

In [2]:
def load_config(file_path="config.yaml"):
    with open(file_path, "r") as file:
        return yaml.safe_load(file)

In [3]:
config = load_config()
db_config = config["database"]

# Load credentials
db_user = db_config["user"]
db_password = db_config["password"]
db_host = db_config["host"]
db_port = db_config["port"]
db_name = db_config["name"]

# DB connection
conn = psycopg2.connect(
    dbname="postgres",
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)
conn.autocommit = True

Check if DB Exists

In [4]:
db_name = "etl_db_final_workshop"
try:
    with conn.cursor() as cur:
        cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
        print(f"Base de datos '{db_name}' creada exitosamente.")
except psycopg2.errors.DuplicateDatabase:
    print(f"La base de datos '{db_name}' ya existe.")
finally:
    conn.close()

La base de datos 'etl_db_final_workshop' ya existe.


Casting datatypes

In [5]:
import pandas as pd
import numpy as np

# Definir tipos de datos optimizados
dtypes = {
    "summary_code": np.uint16,
    "http_code": np.uint16,
    "total_time_ms": np.uint16,
    
    **{col: np.float32 for col in [
        "time_ms_accessory_detector_near_img", "time_ms_accessory_detector_far_img",
        "queue_time_ms_accessory_detector_near_img", "queue_time_ms_accessory_detector_far_img",
        "time_ms_face_detector_near_img", "time_ms_face_detector_far_img",
        "queue_time_ms_face_detector_near_img", "queue_time_ms_face_detector_far_img",
        "time_ms_liqe_near_img", "time_ms_liqe_far_img",
        "queue_time_ms_liqe_near_img", "queue_time_ms_liqe_far_img",
        "time_ms_topiq_near_img", "time_ms_topiq_far_img",
        "queue_time_ms_topiq_near_img", "queue_time_ms_topiq_far_img",
        "time_ms_classic_metrics_near_img", "time_ms_classic_metrics_far_img",
        "time_ms_as_35_selfies_near_img", "time_ms_as_35_selfies_far_img",
        "queue_time_ms_as_35_selfies_near_img", "queue_time_ms_as_35_selfies_far_img",
        "time_ms_ibeta2_crops_near_img", "time_ms_ibeta2_crops_far_img",
        "queue_time_ms_ibeta2_crops_near_img", "queue_time_ms_ibeta2_crops_far_img",
        "time_ms_ibeta2_full_near_img", "time_ms_ibeta2_full_far_img",
        "queue_time_ms_ibeta2_full_near_img", "queue_time_ms_ibeta2_full_far_img",
        "time_ms_ibeta2_clip_near_img", "time_ms_ibeta2_clip_far_img",
        "queue_time_ms_ibeta2_clip_near_img", "queue_time_ms_ibeta2_clip_far_img",
        "time_ms_geometry_check", "queue_time_ms_geometry_check"
    ]},

    **{col: np.float32 for col in [
        "score_liqe_near_img", "score_liqe_far_img",
        "score_topiq_near_img", "score_topiq_far_img",
        "score_sharpness_classic_metrics_near_img", "score_sharpness_classic_metrics_far_img",
        "score_colorfulness_classic_metrics_near_img", "score_colorfulness_classic_metrics_far_img",
        "score_contrast_classic_metrics_near_img", "score_contrast_classic_metrics_far_img",
        "score_brightness_classic_metrics_near_img", "score_brightness_classic_metrics_far_img",
        "score_blur_score_classic_metrics_near_img", "score_blur_score_classic_metrics_far_img",
        "score_svd_score_classic_metrics_near_img", "score_svd_score_classic_metrics_far_img",
        "prob_as_35_selfies_near_img", "prob_as_35_selfies_far_img",
        "prob_ibeta2_crops_near_img", "prob_ibeta2_crops_far_img",
        "prob_ibeta2_full_near_img", "prob_ibeta2_full_far_img",
        "prob_ibeta2_clip_near_img", "prob_ibeta2_clip_far_img",
        "prob_as_heuristics_near_img", "prob_as_heuristics_far_img"
    ]},

    "summary_status": "category",
    "summary_desc": "category",
    "service": "category",
    "convenio": "category",

    "api_hash": str,
    "user_name": str,
    "file_token": str,
    "procesoConvenioGuid": str,
    "documento": str,
    "date": str,
}

Load Data

In [6]:
df = pd.read_csv("../data/processed/anonymized_data.csv", dtype=dtypes)
print(df.columns)
print(df.info())
df.head()

Index(['api_hash', 'user_name', 'file_token', 'service', 'summary_code',
       'summary_status', 'summary_desc', 'http_code', 'total_time_ms', 'date',
       'procesoConvenioGuid', 'convenio', 'documento',
       'img_size_near_img_width', 'img_size_near_img_height',
       'img_size_far_img_width', 'img_size_far_img_height',
       'score_liqe_near_img', 'score_liqe_far_img', 'score_topiq_near_img',
       'score_topiq_far_img', 'score_sharpness_classic_metrics_near_img',
       'score_sharpness_classic_metrics_far_img',
       'score_colorfulness_classic_metrics_near_img',
       'score_colorfulness_classic_metrics_far_img',
       'score_contrast_classic_metrics_near_img',
       'score_contrast_classic_metrics_far_img',
       'score_brightness_classic_metrics_near_img',
       'score_brightness_classic_metrics_far_img',
       'score_blur_score_classic_metrics_near_img',
       'score_blur_score_classic_metrics_far_img',
       'score_svd_score_classic_metrics_near_img',
       '

,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,time_ms_ibeta2_full_near_img,time_ms_ibeta2_full_far_img,queue_time_ms_ibeta2_full_near_img,queue_time_ms_ibeta2_full_far_img,time_ms_ibeta2_clip_near_img,time_ms_ibeta2_clip_far_img,queue_time_ms_ibeta2_clip_near_img,queue_time_ms_ibeta2_clip_far_img,time_ms_geometry_check,queue_time_ms_geometry_check
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08T22:10:36.542487-05:00,...,83.0,83.0,6.0,2.0,9.0,9.0,3.0,2.0,17.0,2.0
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08T22:14:12.653531-05:00,...,40.0,28.0,3.0,1.0,7.0,16.0,2.0,2.0,8.0,1.0
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08T22:16:10.768124-05:00,...,31.0,58.0,3.0,3.0,9.0,14.0,3.0,2.0,5.0,1.0
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08T22:16:14.887930-05:00,...,36.0,60.0,2.0,2.0,11.0,10.0,2.0,1.0,23.0,2.0
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08T22:28:01.054463-05:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Create SQLAlqchemy Engine

In [7]:
# Crear el motor de SQLAlchemy
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

Drop Table if Neccesary 

In [ ]:
# Borrar la tabla 'etl_staging_table_final_workshop' #Staging table
# Borrar la tabla 'etl_transformed_table_final_workshop' #Transformed table
# with engine.connect() as conn:
#     conn.execute(text("DROP TABLE IF EXISTS etl_staging_table_final_workshop"))
#     print("Tabla 'tabla_etl' borrada exitosamente en PostgreSQL.")

Staging Raw Data

In [8]:
db_table_name = "etl_staging_table_final_workshop"
df.to_sql(db_table_name, engine, if_exists='append', index=False)

print(f"DataFrame guardado exitosamente en la tabla {db_table_name} en PostgreSQL.")

DataFrame guardado exitosamente en la tabla etl_staging_table_final_workshop en PostgreSQL.


Loading / Reading Staging Table

In [9]:
db_table_name = "etl_staging_table_final_workshop"
with engine.connect() as conn:
    df_staging = pd.read_sql(f"SELECT * FROM {db_table_name}", conn)

print(df_staging.info())
print(df_staging.describe())
df_staging.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14654 entries, 0 to 14653
Data columns (total 79 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   api_hash                                     14654 non-null  object 
 1   user_name                                    14654 non-null  object 
 2   file_token                                   14654 non-null  object 
 3   service                                      14654 non-null  object 
 4   summary_code                                 14654 non-null  int64  
 5   summary_status                               14654 non-null  object 
 6   summary_desc                                 14654 non-null  object 
 7   http_code                                    14654 non-null  int64  
 8   total_time_ms                                14654 non-null  int64  
 9   date                                         14654 non-null  object 
 10

,api_hash,user_name,file_token,service,summary_code,summary_status,summary_desc,http_code,total_time_ms,date,...,time_ms_ibeta2_full_near_img,time_ms_ibeta2_full_far_img,queue_time_ms_ibeta2_full_near_img,queue_time_ms_ibeta2_full_far_img,time_ms_ibeta2_clip_near_img,time_ms_ibeta2_clip_far_img,queue_time_ms_ibeta2_clip_near_img,queue_time_ms_ibeta2_clip_far_img,time_ms_geometry_check,queue_time_ms_geometry_check
0,02bc5e7,user_name_1,HLqbPi2Mbg,main,1400,ok,antispoofing OK,200,2204,2025-01-08T22:10:36.542487-05:00,...,83.0,83.0,6.0,2.0,9.0,9.0,3.0,2.0,17.0,2.0
1,02bc5e7,user_name_1,DQktwYCPMA,main,1400,ok,antispoofing OK,200,1837,2025-01-08T22:14:12.653531-05:00,...,40.0,28.0,3.0,1.0,7.0,16.0,2.0,2.0,8.0,1.0
2,02bc5e7,user_name_1,5RIpjp7e4Q,main,1400,ok,antispoofing OK,200,1936,2025-01-08T22:16:10.768124-05:00,...,31.0,58.0,3.0,3.0,9.0,14.0,3.0,2.0,5.0,1.0
3,02bc5e7,user_name_1,Csl5ZGg2Jg,main,1400,ok,antispoofing OK,200,1640,2025-01-08T22:16:14.887930-05:00,...,36.0,60.0,2.0,2.0,11.0,10.0,2.0,1.0,23.0,2.0
4,02bc5e7,user_name_1,7RUtj4FLag,main,302,error,blur/haze,200,919,2025-01-08T22:28:01.054463-05:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
